In [1]:
import plumed
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1977) # makes notebook reproducible

In [2]:
kBT=1.2388545199729883*2

In [8]:
# Go to the directory
# !ls

#%cd ~/Documents/work/xerxes/work/HJ/metaD/isoi/
%cd ~/Documents/work/xerxes/work/HJ/metaD/isoii/fourreplicas/

/Users/zhang/Documents/work/xerxes/work/HJ/metaD/isoii/fourreplicas


In [17]:
# Check current directory
import os

os.getcwd()

'/Users/zhang/Documents/work/xerxes/work/HJ/metaD/test'

In [18]:
%ls

COLVAR.0                    coord3.png
COLVAR.1                    coord4.png
COLVAR.2                    coord5.png
COLVAR.3                    demux_traj.py
COLVAR.4                    fep0.png
COLVAR.5                    fep1.png
COLVAR_reweight0            fep2.png
COLVAR_reweight1            fep3.png
COLVAR_reweight2            fep4.png
COLVAR_reweight3            fep5.png
COLVAR_reweight4            fep_isoi_reweighted.png
COLVAR_reweight5            fes.1.dat
COORD_traj0                 fes.10.dat
COORD_traj1                 fes.100.dat
COORD_traj2                 fes.2.dat
COORD_traj3                 fes.20.dat
COORD_traj4                 fes.3.dat
COORD_traj5                 fes.4.dat
GHBFIX.cpp                  fes.5.dat
GHBFIX.o                    fes.50.dat
GHBFIX.so*                  fes.png
HILLS.0                     fes0.dat
HILLS.1                     fes2.dat
HILLS.2                     fes3.dat
HILLS.3                     fes4.dat
HILLS.4                     fes5.dat

In [4]:
colvar_all = []

# This is the main run
for i in range(0,6):
    colvar_all.append(plumed.read_as_pandas("COLVAR_reweight{}".format(i)))


/Applications/PyMOL.app/Contents/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: cannot load PLUMED instance, conversions will not be available
  """


In [33]:
print(colvar_all)

[          time       isoi        h12        h34  metad.bias
0          0.0  37.208996  87.418201  92.196632  191.461819
1          1.0  35.508849  87.418066  79.767734  195.495039
2          2.0  35.040668  91.712261  72.431880  195.223427
3          3.0  35.352920  87.835181  78.258312  195.456313
4          4.0  35.892297  87.657820  82.281016  195.224549
...        ...        ...        ...        ...         ...
9996    9996.0  37.366766  90.957524  89.809195  190.670740
9997    9997.0  33.343004  81.195080  70.635518  192.066106
9998    9998.0  35.801439  84.070604  85.178480  195.345049
9999    9999.0  35.752301  90.052965  78.965024  195.396799
10000  10000.0  33.652000  75.459937  78.420200  192.998059

[10001 rows x 5 columns],           time       isoi        h12        h34  metad.bias
0          0.0  37.208996  87.418201  92.196632  188.288097
1          1.0  36.613809  91.343905  83.903490  190.486657
2          2.0  33.090904  72.298602  77.676078  188.180983
3          3

In [3]:
# Return average and error with bootstrap
# nblocks: number of blocks
# discard: discarded fraction
def analyze(traj,nblocks,discard=0):
    l=int(len(traj)*(1.0-discard))
    # make sure the number of frames is a multiple of nblocks (discard the first few frames)
    l=(l//nblocks)*nblocks
    b=np.array(traj["metad.bias"])
    b-=np.max(b) # avoid overflows
    w=np.exp(b/kBT)[-l:].reshape((nblocks,-1)) # weights
    isA=np.int_(traj["lambda"]==0)[-l:].reshape((nblocks,-1)) # 1 if in A
    isB=np.int_(traj["lambda"]==np.max(traj["lambda"]))[-l:].reshape((nblocks,-1)) # 1 if in B
    B=200 # number of bootstrap iterations
    boot=np.random.choice(nblocks,size=(B,nblocks))
    popA=np.average(isA[boot],axis=(1,2),weights=w[boot])
    popB=np.average(isB[boot],axis=(1,2),weights=w[boot])
    df=np.log(popA/popB) # this is in kBT units
    popA0=np.average(isA,weights=w)
    popB0=np.average(isB,weights=w)
    return np.log(popA0/popB0),np.std(df)

# time-averaged potential, computed averaging over the final 25%
def time_average(hills,t0=0.75):
    n0=int(len(hills)*t0)
    w=np.hstack((np.ones(n0),np.linspace(1.0,0.0,len(hills)-n0)))
    hills=hills.copy()
    hills.height*=w
    return hills

# bias potential computed from hills
# can be used to recompute V(lambda) given the hills file
# can be used on a hills file produced by time_average()
# the result is an array with V for each lambda index
def metad_bias(hills):
    bias=[]
    for i in range(np.max(hills["lambda"]+1)):
        bias.append(np.sum(np.array(hills["lambda"]==i,dtype=int)*hills["height"]/hills["biasf"]*(hills["biasf"]-1)))
    return np.array(bias)

In [7]:
# Read HILLS files
hills_all=[]
for i in range(0,6):
    hills_all.append(plumed.read_as_pandas("HILLS.{}".format(i)))

/Applications/PyMOL.app/Contents/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: cannot load PLUMED instance, conversions will not be available
  after removing the cwd from sys.path.


In [10]:
len(hills_all)

6

In [8]:
# Generate time average result
ave_hills_all = []
for i in range(len(hills_all)):
    ave_hills_all.append(time_average(hills_all[i],0.7))

print(hills_all[0])
print(ave_hills_all[0])

                time       s13       s24  sigma_s13  sigma_s24    height  \
0       2.000000e+00  5.616748  5.756478       0.16       0.16  2.678571   
1       4.000000e+00  5.581061  5.689951       0.16       0.16  2.511398   
2       6.000000e+00  5.563928  5.295557       0.16       0.16  2.667093   
3       8.000000e+00  5.887675  5.666837       0.16       0.16  2.609920   
4       1.000000e+01  5.384134  5.431600       0.16       0.16  2.577396   
...              ...       ...       ...        ...        ...       ...   
499995  9.999920e+05  4.427057  1.070771       0.16       0.16  0.015125   
499996  9.999940e+05  4.336646  1.045048       0.16       0.16  0.015508   
499997  9.999960e+05  4.928701  1.115684       0.16       0.16  0.014953   
499998  9.999980e+05  4.984211  1.120322       0.16       0.16  0.015124   
499999  1.000000e+06  4.511696  1.131418       0.16       0.16  0.014608   

        biasf  
0          15  
1          15  
2          15  
3          15  
4      

In [9]:
# Save new time averaged HILLS files
for i in range(len(ave_hills_all)):
    np.savetxt("A_HILLS.{}".format(i), ave_hills_all[i], delimiter = " ",
              header = "#! FIELDS time s12 s34 sigma_s12 sigma_s34 height biasf\n#! SET multivariate false\n#! SET kerneltype gaussian",
              comments = '',
              fmt = ['%23.9f','%22.13f','%22.13f','%22.1f','%22.1f','%22.13f', '%22u'])


In [ ]:
dd=[]
for i in range(len(colvar_all)):
    df=colvar_all[i]
    # analyze the static potential part
    df1=df.loc[(df['time'] >= 4000)]
    # analyze 3000-4000 ps assuming the final potential
    df2=df.loc[(df['time'] <= 4000) & (df['time'] > 3000) ]
    # analyze 3000-4000 ps assuming the time-averaged potential
    df3=df.loc[(df['time'] <= 4000) & (df['time'] > 3000) ].copy()
    df3["metad.bias"]=metad_bias(time_average(hills_all[i]))[np.int_(df3["lambda"])]   
    dd.append((
        analyze(df1,len(df1)//2500),
        analyze(df2,len(df2)//2500),
        analyze(df3,len(df3)//2500))
    )
dd=np.array(dd)
# dd[i,j,k]
# i=0..19 (iteration)
# j=0..2 (method)
# k=0..1 (average and stddev)


In [13]:
!cd ~